In [41]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import sklearn as sk
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
import os
from keras.utils import np_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import classification_report
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence

from keras.regularizers import l2

from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from sklearn.model_selection import train_test_split

from keras.callbacks import LambdaCallback
from sklearn import svm


from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from datetime import datetime

In [42]:
dataname = "rawdata_100.pkl"
modelname = 'SVM.h5'
max_features = 10000

dim_output = 2

size_batch = 10
num_epoch = 100
l2_lambda = 0.00001

In [43]:
data = pd.read_pickle(dataname)

stemmer = SnowballStemmer("english")
tokenizer = RegexpTokenizer("[a-z']+")

def tokenize(text):
    tokens = tokenizer.tokenize(text)
    return [stemmer.stem(t) for t in tokens] 

def get_tf(data, use_idf, max_df=1.0, min_df=1, ngram_range=(1,1)):
    if use_idf:
        m = TfidfVectorizer(max_df=max_df, min_df=min_df, stop_words='english', ngram_range=ngram_range, tokenizer=tokenize, max_features = max_features)
    else:
        m = CountVectorizer(max_df=max_df, min_df=min_df, stop_words='english', ngram_range=ngram_range, tokenizer=tokenize)
    
    d = m.fit_transform(data)
    return m, d

# tf_m, tf_d = get_tf(data['reviewText'], use_idf=False, max_df=0.90, min_df=10)
tfidf_m, tfidf_d = get_tf(data['reviewText'], use_idf=True, max_df=0.90, min_df=10)

In [44]:
def getData(filename):
    print("---------------")
    print("| Getting data...")
    print("---------------")
    
    data_X = tfidf_d
#     data_Y = np_utils.to_categorical(data['overall'], dim_output)
    data_Y = data.overall
    return data_X, data_Y

In [45]:
def splitData(x, y):
    X_trains, X_test, Y_trains, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    X_train, X_validation, Y_train, Y_validation = train_test_split(X_trains, Y_trains, test_size=0.2, random_state=42)
    return X_train, X_validation, X_test, Y_train, Y_validation, Y_test

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
def getParameter(x_train, y_train):
    
    str(datetime.now())
    
    tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10]}]
    score='accuracy'

    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(), tuned_parameters, cv=2, scoring=score, verbose = True)
    clf.fit(x_train, y_train)

    print("Best parameters set found on development set:")
    print(clf.best_params_)
    
    str(datetime.now())

In [47]:
from sklearn.metrics import accuracy_score
def train(x_train, y_train):
    print("---------------")
    print("| Training...")
    print("---------------")
    
    print(str(datetime.now()))
    
#     model=svm.SVC(gamma=0.01,C=100.,decision_function_shape='ovo')
    model = svm.LinearSVC(verbose = True)
    model.fit(x_train, y_train)
    
    print(str(datetime.now()))
    
    return model

In [48]:
def test(model, x_test, y_test):
    print("---------------")
    print("| Testing...")
    print("---------------")
    
    print(str(datetime.now()))

    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    print(classification_report(y_test,y_pred))
    
    print(str(datetime.now()))

In [51]:
# def main():
X, Y = getData(dataname)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model = train(X_train,Y_train)
test(model, X_test, Y_test)

---------------
| Getting data...
---------------
---------------
| Training...
---------------
2018-06-17 11:59:27.822552
[LibLinear]2018-06-17 11:59:28.910185
---------------
| Testing...
---------------
2018-06-17 11:59:28.911201
0.8663943990665111
             precision    recall  f1-score   support

          0       0.87      0.87      0.87     10330
          1       0.86      0.87      0.87     10238

avg / total       0.87      0.87      0.87     20568

2018-06-17 11:59:28.923150


In [52]:
# main()

### Adversasial

In [53]:
def getX(x_test):
    return tfidf_m.fit_transform(x_test)

In [54]:
filename_adv = "data_adversial_wordnet.pkl"
data = pd.read_pickle(filename_adv)
X_test_adv = tfidf_m.fit_transform(data['reviewText'])
Y_test_adv = data['overall']
test(model, X_test_adv, Y_test_adv)

---------------
| Testing...
---------------
2018-06-17 12:02:19.329849


ValueError: X has 9408 features per sample; expecting 8637

In [55]:
X_test = np.load('X_test.npy')
X_test_adv_responding = getX(X_test)
Y_test_adv_responding = np.load('Y_test.npy')

In [56]:
Y_pred_adv_responding = model.predict(X_test_adv_responding)
accuracy = accuracy_score(Y_test_adv_responding, Y_pred_adv_responding)
print(accuracy)
print(classification_report(Y_test_adv_responding, y_pred_adv_responding))

ValueError: X has 4945 features per sample; expecting 8637

In [ ]:
test(model, X_test_adv_responding, Y_test_adv_responding)